In [5]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertPreTrainedModel, BertModel

from transformers import AutoConfig, AutoTokenizer

In [6]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

In [9]:
train_df = pd.read_csv('data/train.txt', sep=';')
test_df = pd.read_csv('data/test.txt', sep=';')
val_df = pd.read_csv('data/val.txt', sep=';')

In [12]:
train_df.shape, test_df.shape, val_df.shape

((15999, 2), (1999, 2), (1999, 2))

In [13]:
train_df.columns = ['sentence', 'emotion']
test_df.columns = ['sentence', 'emotion']
val_df.columns = ['sentence', 'emotion']

In [19]:
# get max len of sentences
def max_len(data):
    return data['sentence'].apply(lambda x: len(x.split())).max()

max_lens = [max_len(train_df), max_len(test_df), max_len(val_df)]
max_lens

[66, 61, 61]

In [23]:
MODEL_OUT_DIR = 'model_output'
TRAIN_FILE_PATH = 'data/train.txt'
VALID_FILE_PATH = 'data/val.txt'
TEST_FILE_PATH = 'data/test.txt'

## Model Configurations
MAX_LEN_TRAIN = 68
MAX_LEN_VALID = 68
MAX_LEN_TEST = 68
BATCH_SIZE = 160
LR = 1e-5
NUM_EPOCHS = 10
NUM_THREADS = 1  ## Number of threads for collecting dataset
MODEL_NAME = 'bert-base-uncased'
LABEL_DICT = {'joy':0, 'sadness':1, 'anger':2, 'fear':3, 'love':4, 'surprise':5}

if not os.path.isdir(MODEL_OUT_DIR):
    os.makedirs(MODEL_OUT_DIR)